In [ ]:
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pyautogui
from PIL import ImageGrab

### 1. Test Screen Capture

- Get All Window

In [ ]:
windows = pyautogui.getAllTitles()

windows = set([w for w in windows if w != ''])

windows

- Screen Capture

In [ ]:
window_name = 'Zoom Meeting'

In [ ]:
def screen_capture(window_name= 'Zoom Meeting'):
    if window_name not in pyautogui.getAllTitles():
        print("Could not find window name '%s'" % window_name)
        return None
    
    fw = pyautogui.getWindowsWithTitle(window_name)[0]
    fw.maximize()
    pyautogui.click(fw.center)
    fw.activate()

    time.sleep(1)

    sct = ImageGrab.grab() 
    img = np.array(sct)
    
    return img

In [ ]:
img = screen_capture(window_name= 'Zoom Meeting')

- Show captured Screen

In [ ]:
plt.figure(figsize=(16, 10))
plt.imshow(img)

- Test Preprocess Screen Capture

In [ ]:
def preprocess(img):
    face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    
    img_list = []
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    faces = face_cascade.detectMultiScale(img_gray, 1.3, 5)
    for (x, y, w, h) in faces:
        img_face = img_gray[y:y+h, x:x+w]  # crop face image 
        img_resize = cv2.resize(img_face, (100, 100)) # resize to 100 x 100 pixel
        img_list.append(img_resize)
    return img_list, faces

In [ ]:
img_list = preprocess(img)[0]

In [ ]:
for face in img_list:
    plt.imshow(face, cmap="gray")

### 2. Load Model PCA

In [ ]:
import pickle

def read_model(filename, path=""):
    with open(os.path.join(path, filename), 'rb') as in_name:
        model = pickle.load(in_name)
        return model

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = read_model("pca_model.pkl")

In [ ]:
# convert each detected face to 1D array feature vector
img_list_flatten = [img.flatten() for img in img_list]


# apply PCA to each 1D array feature vector
img_list_pca = pca.transform(img_list_flatten)

### 3. load model k-NN

In [ ]:
labels = ['Colin_Powell', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Yunus']

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

- load model

In [ ]:
knn_model = read_model("k-NN_scikit_model.pkl")

- predict data

In [ ]:
ids = knn_model.predict(img_list_pca)

In [ ]:
label_output = [labels[i] for i in ids]

print("Predicted Name : %s " % label_output[0])

In [ ]:
proba = knn_model.predict_proba(img_list_pca)

confidence = np.max(proba)

print("Confidence : %.3f" % confidence)

### 4. load model SVM

In [ ]:
labels = ['Colin_Powell', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Yunus']

In [ ]:
from sklearn.svm import SVC

- load model

In [ ]:
model_svm = read_model("SVM_scikit_model.pkl")

- predict data

In [ ]:
ids = model_svm.predict(img_list_pca)

In [ ]:
label_output = [labels[i] for i in ids]

print("Predicted Name : %s " % label_output[0])

In [ ]:
proba = model_svm.predict_proba(img_list_pca)

confidence = np.max(proba)

print("Confidence : %.3f" % confidence)

___
# Test using Realtime Capture

In [ ]:
def draw_ped(img, label, x0, y0, xt, yt, color=(255,127,0), text_color=(255,255,255)):

    (w, h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(img,
                  (x0, y0 + baseline),  
                  (max(xt, x0 + w), yt), 
                  color, 
                  2)
    cv2.rectangle(img,
                  (x0, y0 - h),  
                  (x0 + w, y0 + baseline), 
                  color, 
                  -1)  
    cv2.putText(img, 
                label, 
                (x0, y0),                   
                cv2.FONT_HERSHEY_SIMPLEX,     
                0.5,                          
                text_color,                
                1,
                cv2.LINE_AA) 
    return img

In [ ]:
def screen_capture(window_name= 'Zoom Meeting'):
    if window_name not in pyautogui.getAllTitles():
        return None
    try :
        fw = pyautogui.getWindowsWithTitle(window_name)[0]
        fw.maximize()
        pyautogui.click(fw.center)
        fw.activate()
    except :
        pass
    
    time.sleep(1)

    sct = ImageGrab.grab() 
    img = np.array(sct)
    
    return img

In [ ]:
def preprocess(img):
    face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    
    img_list = []
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    faces = face_cascade.detectMultiScale(img_gray, 1.3, 5)
    for (x, y, w, h) in faces:
        img_face = img_gray[y:y+h, x:x+w]  # crop face image 
        img_resize = cv2.resize(img_face, (100, 100)) # resize to 100 x 100 pixel
        img_list.append(img_resize)
    return img_list, faces

In [ ]:
labels = ['Colin_Powell', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Yunus']

model_svm = read_model("SVM_scikit_model.pkl")


while True:
    # get data     
    img = screen_capture(window_name= 'Zoom Meeting')
    if img is None :
        print("[INFO] could not find window !")
        break
    
    # preprocessing    
    img_list, face_coords = preprocess(img)
    if len(img_list) < 1 :
        print("[INFO] could not find face!")
    else : 
        # convert each detected face to 1D array feature vector
        img_list_flatten = [img.flatten() for img in img_list]

        # apply PCA to each 1D array feature vector
        img_list_pca = pca.transform(img_list_flatten)

        # predict data using SVM    
        ids = model_svm.predict(img_list_pca)
        label_output = [labels[i] for i in ids]

        proba = model_svm.predict_proba(img_list_pca)
        confidence = [np.max(p) for p in proba]

        for i in range(len(ids)):
            plt.imshow(img_list[i], cmap="gray")
            title = "Name : %s (%.2f%%)" % (label_output[i], (confidence[i]*100))
            plt.title(title)
            plt.show()
            
            x, y, w, h = face_coords[i]
            x0, y0, xt, yt = x, y, x+w, y+h
            img = draw_ped(img, title, x0, y0, xt, yt)
            
    cv2.imshow("Facerecognition", img[:,:,::-1])
    if cv2.waitKey(3000) == ord('q'):
        break

cv2.destroyAllWindows()

### Add Toast WIndows Notification

In [ ]:
! pip install win10toast

In [ ]:
from win10toast import ToastNotifier
toaster = ToastNotifier()
toaster.show_toast("Sample Notification","Python is awesome!!!")

- apply to realtime capture

In [ ]:
labels = ['Colin_Powell', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Yunus']

model_svm = read_model("SVM_scikit_model.pkl")

toaster = ToastNotifier()

while True:
    # get data     
    img = screen_capture(window_name= 'Zoom Meeting')
    if img is None :
        print("[INFO] could not find window !")
        break
    
    # preprocessing    
    img_list, face_coords = preprocess(img)
    if len(img_list) < 1 :
        print("[INFO] could not find face!")
    else : 
        # convert each detected face to 1D array feature vector
        img_list_flatten = [img.flatten() for img in img_list]

        # apply PCA to each 1D array feature vector
        img_list_pca = pca.transform(img_list_flatten)

        # predict data using SVM    
        ids = model_svm.predict(img_list_pca)
        label_output = [labels[i] for i in ids]

        proba = model_svm.predict_proba(img_list_pca)
        confidence = [np.max(p) for p in proba]

        for i in range(len(ids)):
            plt.imshow(img_list[i], cmap="gray")
            title = "Name : %s (%.2f%%)" % (label_output[i], (confidence[i]*100))
            plt.title(title)
            plt.show()
            
            x, y, w, h = face_coords[i]
            x0, y0, xt, yt = x, y, x+w, y+h
            img = draw_ped(img, title, x0, y0, xt, yt)
            
            toaster.show_toast("Attendance Systems", title)
            
    cv2.imshow("Facerecognition", img[:,:,::-1])
    if cv2.waitKey(3000) == ord('q'):
        break

cv2.destroyAllWindows()